In [90]:
# MODULE IMPORTS
import pandas as pd
import os
import json
import numpy as np

In [91]:
data = pd.read_excel(f"{os.getcwd()[:-7]}datasets/aspe-uninsured-estimates-by-county.xlsx", "All Uninsured (%)")
data['FIPS Code'] = data['FIPS Code'].astype(str)
data.replace("**", np.NAN, inplace=True)

with open(f"{os.getcwd()[:-7]}/project/webapp/src/components/counties.json", 'r', encoding='cp1252') as loading:
    counties = json.load(loading)

data.head()

,State Name,FIPS Code,County Name,Total Non-Elderly Population (Excluding Undocumented),Uninsured Population (Excluding Undocumented),Percent Uninsured,% HIU Income < 100% FPL,% HIU Income 100-138% FPL,% HIU Income 139-249% FPL,% HIU Income 250-400% FPL,...,% College Grad,% No English Speaking Adults in HH,% English Spoken in HH,% Spanish Spoken in HH,% Chinese Spoken in HH,% Korean Spoken in HH,% Vietnamese Spoken in HH,% Tagalog Spoken in HH,% Russian Spoken in HH,% Other Language Spoken in HH
0,Alabama,1001,Autauga AL,46000,4300,0.094047,0.523231,0.163396,0.132692,0.120478,...,0.066593,0.020697,0.978894,0.021106,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,1003,Baldwin AL,171200,15600,0.091068,0.450606,0.115942,0.152891,0.143406,...,0.159999,0.039048,0.952021,0.039048,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,1005,Barbour AL,22100,2700,0.120872,0.460045,0.105182,0.180687,0.224628,...,0.035638,0.052478,0.931596,0.061232,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,1007,Bibb AL,16800,2700,0.160059,0.539555,0.140007,0.123993,0.100681,...,0.064132,NaN,0.986084,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,1009,Blount AL,49500,6400,0.128740,0.547263,0.101158,0.104732,0.146646,...,0.044229,0.094751,0.903658,0.096342,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# Add data from df to counties
for feature in counties['features']:
    geo_id = feature['properties']['GEO_ID'].split("US")[-1]
    county_data = data.loc[data['FIPS Code'] == geo_id.lstrip("0")].squeeze()
    
    if county_data.empty:
        county_data = pd.Series(index=data.columns)

    feature['properties']['pct_uninsured'] = county_data.loc["Percent Uninsured"]
    feature['properties']['finance_data'] = {
        "Below FPL": county_data.loc["% HIU Income < 100% FPL"],
        "100-138% FPL": county_data.loc["% HIU Income 100-138% FPL"],
        "139-249% FPL": county_data.loc["% HIU Income 139-249% FPL"],
        "250-400% FPL": county_data.loc["% HIU Income 250-400% FPL"],
        "Above 400% FPL": county_data.loc["% HIU Income 400% FPL"],
        "Unknown": 1 - (county_data.loc['% HIU Income < 100% FPL':'% HIU Income 400% FPL'].sum())
    }
    feature['properties']['age_data'] = {
        "Under 18": county_data.loc["% Age 0-18"],
        "19-34": county_data.loc["% Age 19-34"],
        "35-49": county_data.loc["% Age 35-49"],
        "50-64": county_data.loc["% Age 50-64"],
        "Unknown": 1 - (county_data.loc['% Age 0-18':'% Age 50-64'].sum())
    }
    feature['properties']['race_data'] = {
        'Spanish/Hispanic/Latino': county_data.loc["% Spanish/Hispanic/Latino Origin"],
        'White': county_data.loc["% White Non-Latino"],
        'Black': county_data.loc["% Black Non-Latino"],
        'Asian/Pacific Islander': county_data.loc["% Asian / Native-Hawaiian / Pac Islander"],
        'Native American': county_data.loc["% American Indian / Alaska Native"],
        'Multi-racial or Other': county_data.loc["% Multi-racial or Other"],
        "Unknown": 1 - (county_data.loc["% Spanish/Hispanic/Latino Origin":"% Multi-racial or Other"].sum())
    }
    feature['properties']['social_data'] = {
        '% Male': county_data.loc['% Male'],
        '% Female': county_data.loc['% Female'],
        '% Female': county_data.loc['% Female'],
        '% Married': county_data.loc['% Married'],
        '% Child in Family': county_data.loc['% Child in Family'],
        '% Full-time Worker in Family': county_data.loc['% Full-time Worker in Family'],
        '% SNAP Recipient': county_data.loc['% SNAP Recipient'],
        '% With Disability': county_data.loc['% With a disability'],
    }
    feature['properties']['education_data'] = {
        '% Less than High School': county_data.loc['% Less than High School'],
        '% High School Diploma': county_data.loc['% High School Diploma'],
        '% College Grad': county_data.loc['% College Grad'],
        'Unknown': 1 - (county_data.loc['% Less than High School':'% College Grad'].sum())
    }

C:\Users\Simi\AppData\Local\Temp/ipykernel_29304/4221282136.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  county_data = pd.Series(index=data.columns)


In [97]:
# convert Nan to null and saving the file
import simplejson

with open(f"{os.getcwd()[:-7]}/project/webapp/src/components/data.json", 'w', encoding='cp1252') as path:
    simplejson.dump(counties, path, ignore_nan=True)